# 🌊 **Klasifikasi Tsunami Menggunakan Random Forest dan Logistic Regression**

## 🎯 **Tujuan Proyek**
Proyek ini bertujuan untuk **membangun dan membandingkan performa dua algoritma klasifikasi** dalam memprediksi kemungkinan terjadinya **tsunami** berdasarkan data parameter gempa bumi.  
Dataset yang digunakan adalah **`earthquake_data_tsunami.csv`**, yang berisi data numerik seperti magnitudo, kedalaman, lokasi, serta label target `tsunami` (0 = tidak, 1 = ya).

Proyek ini disusun untuk memenuhi **Ujian Tengah Semester (UTS)** mata kuliah **Pembelajaran Mesin**, dengan fokus pada penerapan **data preprocessing**, **pipeline modeling**, **feature selection**, serta **evaluasi model klasifikasi numerik** sesuai pedoman yang diberikan dosen pengampu.

---

## ⚙️ **Tujuan Analisis**
1. Melakukan **pembersihan data** dengan memeriksa nilai kosong, duplikat, dan outlier.  
2. Melakukan **preprocessing** data numerik serta memisahkan fitur (`X`) dan target (`y`).  
3. Membangun **dua model klasifikasi utama**:
   - **Tree-Based Model** → Random Forest  
   - **Linear-Based Model** → Logistic Regression  
4. Melakukan **eksperimen pembelajaran mesin** dengan:
   - **Dua metode penskalaan (scaling):** `StandardScaler` dan `MinMaxScaler`  
   - **Dua metode seleksi fitur (feature selection):** `SelectKBest` dan `SelectPercentile`  
5. Menggunakan **GridSearchCV** dengan **StratifiedKFold Cross Validation** untuk mencari parameter optimal.  
6. Membandingkan hasil evaluasi model berdasarkan **akurasi, presisi, recall, dan F1-score** serta menampilkan **visualisasi confusion matrix**.

---

## 🌳 **Alasan Pemilihan Algoritme**
- **Random Forest (Tree-Based)**  
  Algoritme ini kuat dalam menangani hubungan non-linear antar fitur, relatif tahan terhadap outlier, dan mampu mengukur tingkat kepentingan fitur secara langsung.

- **Logistic Regression (Linear-Based)**  
  Model ini sederhana dan efisien untuk data numerik, serta memberikan interpretasi yang jelas terhadap probabilitas kelas. Logistic Regression juga menjadi baseline penting untuk membandingkan performa pendekatan non-linear seperti Random Forest.

---

## 🧩 **Tahapan Eksperimen**
1. **Data Understanding & Cleaning**  
   Mengecek struktur data, nilai kosong, duplikat, dan distribusi fitur.  
2. **Feature Selection & Encoding**  
   Menentukan kolom fitur (X) dan target (y) serta memastikan seluruh data numerik siap diproses.  
3. **Train-Test Split**  
   Membagi data menjadi data latih dan uji (contoh: 80:20 atau 75:25 atau 70:30) dengan parameter `random_state` sesuai dua digit NPM terbesar anggota kelompok.  
4. **Pipeline Construction**  
   Membangun pipeline untuk kedua model, berisi tahap scaling → feature selection → classifier.  
5. **Model Training & Tuning**  
   Melakukan pencarian parameter terbaik menggunakan **GridSearchCV** dan validasi lipat (**StratifiedKFold**).  
6. **Evaluation & Visualization**  
   Membandingkan hasil evaluasi model menggunakan metrik klasifikasi serta menampilkan **Confusion Matrix** dan **Classification Report**.

---

## 📊 **Hasil yang Diharapkan**
- Didapatkan **model terbaik** dengan skor F1 dan akurasi tertinggi.  
- Teridentifikasi **fitur-fitur paling relevan** dalam menentukan potensi tsunami.  
- Diperoleh **perbandingan performa** antara model Random Forest dan Logistic Regression sebagai dasar analisis efektivitas algoritme berbasis pohon dan linear.

---


In [1]:
# ============================================================
# 🧮 IMPORT LIBRARY — Pengolahan Data, Modeling, & Evaluasi
# ============================================================

# 📦 Manipulasi Data

import numpy as np
import pandas as pd  

# 📊 Visualisasi
import matplotlib.pyplot as plt

# 🔧 Pembagian data & pencarian hyperparameter
from sklearn.model_selection import train_test_split            
from sklearn.model_selection import StratifiedKFold, GridSearchCV

# ⚙️ Pra-pemrosesan & Seleksi Fitur
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.feature_selection import SelectKBest, SelectPercentile

# 🤖 Model Klasifikasi
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression

# 🧩 Pipeline & Utilitas
from sklearn.pipeline import Pipeline
from sklearn.base import clone
import time

# 📈 Evaluasi Model
from sklearn.metrics import (
    confusion_matrix,
    ConfusionMatrixDisplay,
    classification_report
)

# **Loading Data**

In [3]:
df_tsunami = pd.read_csv('dataset/earthquake_data_tsunami.csv',header=0)

df_tsunami.head()

,magnitude,cdi,mmi,sig,nst,dmin,gap,depth,latitude,longitude,Year,Month,tsunami
0,7.0,8,7,768,117,0.509,17.0,14.000,-9.7963,159.596,2022,11,1
1,6.9,4,4,735,99,2.229,34.0,25.000,-4.9559,100.738,2022,11,0
2,7.0,3,3,755,147,3.125,18.0,579.000,-20.0508,-178.346,2022,11,1
3,7.3,5,5,833,149,1.865,21.0,37.000,-19.2918,-172.129,2022,11,1
4,6.6,0,2,670,131,4.998,27.0,624.464,-25.5948,178.278,2022,11,1


In [4]:
# Info cepat tentang kolom & tipe datanya
print("Jumlah baris, kolom:", df_tsunami.shape)    
print("\nTipe data:")
print(df_tsunami.dtypes)

Jumlah baris, kolom: (782, 13)

Tipe data:
magnitude    float64
cdi            int64
mmi            int64
sig            int64
nst            int64
dmin         float64
gap          float64
depth        float64
latitude     float64
longitude    float64
Year           int64
Month          int64
tsunami        int64
dtype: object


### **Pembersihan Data (Bagian 1): Buang Kolom Tak Perlu**
- Kolom `Year` dan `month` tidak dibutuhkan untuk pemodelan (tidak dibutuhkan untuk prediksi), jadi kita hapus.


In [8]:
# 1) Hapus kolom 'year' dan `month`
df_tsunami2 = df_tsunami.drop(columns=['Year', 'Month'], errors='ignore')
df_tsunami2.head()

,magnitude,cdi,mmi,sig,nst,dmin,gap,depth,latitude,longitude,tsunami
0,7.0,8,7,768,117,0.509,17.0,14.000,-9.7963,159.596,1
1,6.9,4,4,735,99,2.229,34.0,25.000,-4.9559,100.738,0
2,7.0,3,3,755,147,3.125,18.0,579.000,-20.0508,-178.346,1
3,7.3,5,5,833,149,1.865,21.0,37.000,-19.2918,-172.129,1
4,6.6,0,2,670,131,4.998,27.0,624.464,-25.5948,178.278,1


### **Pembersihan Data (Bagian 2): Cek & Tangani Missing Value**
- Kita cek data **null/kosong/NaN** per kolom.  
- Kolom **`texture_mean`** memiliki beberapa nilai kosong dan diisi dengan **median** (aman saat kita belum tahu distribusinya).


In [9]:
# 1) Cek jumlah nilai kosong per kolom
print("Jumlah nilai kosong per kolom:\n", df_tsunami2.isnull().sum()) 

Jumlah nilai kosong per kolom:
 magnitude    0
cdi          0
mmi          0
sig          0
nst          0
dmin         0
gap          0
depth        0
latitude     0
longitude    0
tsunami      0
dtype: int64


In [20]:
# 2) Contoh pengisian: gunakan median untuk kolom 'texture_mean' (jika ada)
median_chole = df_tsunami2['cdi'].median()
df_tsunami2['cdi'] = df_tsunami2['cdi'].fillna(median_chole)
print("\nMedian cdi:", median_chole)
#karena dataset yang kami miliki tidak memiliki data kosong/NaN/NULL maka kami mengambil salah satu contoh penangan jika ada data kosong di cdi


Median cdi: 5.0


In [ ]:
# 3) Validasi ulang
print("\nSetelah inputasi, nilai kosong per kolom:\n", df_tsunami2.isnull().sum())


Setelah imputasi, nilai kosong per kolom:
 magnitude    0
cdi          0
mmi          0
sig          0
nst          0
dmin         0
gap          0
depth        0
latitude     0
longitude    0
tsunami      0
dtype: int64


### **Pembersihan Data (Bagian 3): Cek & Hapus Duplikat**
- Data yang **kembar** dapat merusak evaluasi model.
- Kita cek duplikat lalu **drop** agar setiap baris unik.


In [ ]:
before = df_tsunami2.shape
dupes = df_tsunami2[df_tsunami2.duplicated(keep=False)]
print(f"Jumlah baris duplikat (terhitung ganda): {dupes.shape[0]}")
df_tsunami3 = df_tsunami2.drop_duplicates(keep='first')
print("Bentuk data sebelum/ setelah hapus duplikat:", before, "->", df_tsunami3.shape)

Jumlah baris duplikat (terhitung ganda): 0
Bentuk data sebelum/ setelah hapus duplikat: (782, 11) -> (782, 11)


### **Pembersihan Data (Bagian 4): Cek Outlier**
- Data yang **Outlier** dapat merusak evaluasi model.
- Kenapa tidak dihapus? karena itu adalah fenomena nyata dan **Penting** untuk klasifikasi.


In [17]:
numeric_cols = df_tsunami3.select_dtypes(include=np.number).columns

print("Pengecekan outlier untuk setiap fitur numerik:\n")
for col in numeric_cols:
    Q1 = df_tsunami3[col].quantile(0.25)
    Q3 = df_tsunami3[col].quantile(0.75)
    IQR = Q3 - Q1
    outliers = ((df_tsunami3[col] < (Q1 - 1.5 * IQR)) | (df_tsunami3[col] > (Q3 + 1.5 * IQR))).sum()
    print(f"{col}: {outliers} outlier")

Pengecekan outlier untuk setiap fitur numerik:

magnitude: 37 outlier
cdi: 0 outlier
mmi: 1 outlier
sig: 73 outlier
nst: 0 outlier
dmin: 61 outlier
gap: 48 outlier
depth: 139 outlier
latitude: 0 outlier
longitude: 0 outlier
tsunami: 0 outlier


In [22]:
# Tangani outlier depth dengan batas atas
df_tsunami3['depth'] = np.where(df_tsunami3['depth'] > 700, 700, df_tsunami3['depth'])

In [ ]:
# Menampilkan Data Outlier
plt.figure(figsize=(8,4))
plt.boxplot([df_tsunami['magnitude'], df_tsunami['depth'], df_tsunami['sig'], df_tsunami['mmi'], df_tsunami['cdi']],
            labels=['magnitude','depth','sig','mmi','cdi'])
plt.title("Boxplot Deteksi Outlier pada Fitur Numerik")
plt.ylabel("Nilai")
plt.show()

### **Pembagian Data: Train/Test Split**

- **X**: semua fitur kecuali label target.  
- **y**: kolom target, yaitu `tsunami` (1 = Ya)
 atau (0 = Tidak )
- Kita pakai **30%** data untuk **test** dan sisanya untuk **train**.


In [12]:
# Menentukan X sebagai fitur (semua kolom kecuali diagnosis)
X = df_tsunami3.drop(columns=['tsunami'])

# Menentukan y sebagai target (kolom diagnosis)
y = df_tsunami3['tsunami']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state = 98, stratify=y
)                                                          

print("Ukuran X_train, X_test:", X_train.shape, X_test.shape)


Ukuran X_train, X_test: (625, 10) (157, 10)
